**Brainstorm**:
- Paradigm 
- Example of encapsulation
- Quick SOLID + encapsulation & inheritance
- Match definition game (cognitive map of OOP): class, object, instance, interface/protocol, attribute, method, class attribute, property, public/private/protected
- str/repr + state
- Inheritance exercise
- Object creation:
    * Constructor
    * Factory method (as class method or not)
    * Builder
    * Prototype
- Generic typing?

# Object Oriented Programming and object creation

:hourglass: 3h



OOP
---
:hourglass: 


### The OOP paradigm

> A (programming) **paradigmn** is a way to think about, approach and solve a problem. It defines the (conceptual) primitives in which to think in order to create the solution.

There are several broad families of paradigms:
- Imperative: dictates how the *state* evolves
    * Procedural: the primivites are procedures (~functions);
    * OOP: the primivites are objects exchanging messages;
- Declarative: expresses the relationship between primitives

The example below illustrates the difference between the procedural and OOP paradigm.

In [2]:
# Procedural

def create_car(position=0, speed=0):
    return {"position": position, "speed": speed}

def accelerate_car(car):
    car["speed"] += 1

def decelerate_car(car):
    car["speed"] -= 1

def move_car(car):
    car["position"] += car["speed"]

car = create_car(speed=1)
accelerate_car(car)
move_car(car)
print("Car position:", car["position"])

Car position: 2


In [5]:
# OOP

class Car:
    def __init__(self, position=0, speed=0) -> None:
        self._position = position
        self._speed = speed

    def accelerate(self):
        self._speed += 1

    def decelerate(self):
        self._speed -= 1

    def move(self):
        self._position += self._speed

    def print_position(self):
        print("Car position:", self._position)

car = Car(speed=1)
car.accelerate()
car.move()
car.print_position()


Car position: 2


At first glance there are not many differences between the two approaches. However, those are fundamental!

The car class **encapsulate** both the data (ie. `position` and `speed`) as well as the behavior (`accelerate`, `decelerate`, `move` and `print_position`). This has a few advantages:
- *maintainability*: the behavior of the `car` sits with its data: you need only to edit the code in one place;
- *abstraction*: the user does not need know (and cannot mess up with) the details: it only sends messages via the methods;
- *conceptualization*: the notion of objects makes it easy to think in business terms;
- *inheritance*: the way the code is written makes it easy to implement inheritance a structured way.


Let's see an example of inheritance:

In [10]:
# Inheritance

class LimitedCar(Car):
    def __init__(self, max_speed, position=0, speed=0) -> None:
        super().__init__(position, speed)
        self._max_speed = max_speed

    def accelerate(self):
        if self._speed < self._max_speed:
            return super().accelerate()
        else:
            print("Max speed reached")
    

car = LimitedCar(max_speed=4, speed=3)
car.accelerate()
car.accelerate()

Max speed reached


### Cognitive map of OOP

What is the difference between the following pairs of concepts:
- class and object
- object and instance
- class attribute and attribute
- attribute and property
- interface/protocol and abstract class
- method and function
- class method and method
- public method/attribute and private method/attribute
- private method/attribute and protected method/attribute

### The SOLID principles  :skull: 

People generally think of the OOP paradigm along the lines of the SOLID acrynom:
- **S**ingle-responsibility principle
- **O**pen-closed principle
- **L**iskov substitution principle
- **I**nterface segregation principle
- **D**ependency inversion principle

The general idea is that a class should have a clear purpose, shared with its subclasses and the details should not matter for users.


See for more https://en.wikipedia.org/wiki/SOLID.

Data classes
------------

### Usage 
When writing code, some classes will naturally tend to have lots of methods and do big computations. Sometimes, however, you will just need a convenient way to store data, possibly with a couple of methods. For that data classes are great and efficient:

In [19]:
from dataclasses import dataclass  # import the dataclass decorator

import datetime as dt
from typing import Optional

@dataclass  # annotate the class as being a dataclass
class Person:
    first_name: str
    last_name: str
    birth_date: dt.date
    likes_python: bool = True

    def get_age(self, at_date: Optional[dt.date] = None) -> int:
        if at_date is None:
            at_date = dt.date.today()

        return at_date.year - self.birth_date.year


p = Person("Guido", "van Rossum", dt.date(1956, 1, 31))
print(f"Age: {p.get_age()} ({p})")

Age: 68 (Person(first_name='Guido', last_name='van Rossum', birth_date=datetime.date(1956, 1, 31), likes_python=True))
<bound method __create_fn__.<locals>.__init__ of Person(first_name='Guido', last_name='van Rossum', birth_date=datetime.date(1956, 1, 31), likes_python=True)>


The `dataclass` annotation will generate the `__init__` method (as well as other things). You only need to declare and type the attribute the *instance* will have within the *class* body. Note how the `likes_python` default value was passed to the instance.

There are a few ways to customize the dataclass:
- you can have complex (ie. more complex than a default value) initialization; see `dataclasses.Field` and the `__post_init__` method;
- you can customize whether the instances are mutable, comparable, representable and hashable; see the full documentation at https://docs.python.org/3.9/library/dataclasses.html



### Dataclass and NamedTuple

An alternative to dataclasses are named tuples, which can be used in essentially the same manner:

In [20]:
from typing import NamedTuple

import datetime as dt
from typing import Optional

class Person(NamedTuple):  # inherit from NamedTuple
    first_name: str
    last_name: str
    birth_date: dt.date
    likes_python: bool = True

    def get_age(self, at_date: Optional[dt.date] = None) -> int:  # Defining behavior on NamedTuple is discouraged
        if at_date is None:
            at_date = dt.date.today()

        return at_date.year - self.birth_date.year


p = Person("Guido", "van Rossum", dt.date(1956, 1, 31))
print(f"Age: {p.get_age()} ({p})")

Age: 68 (Person(first_name='Guido', last_name='van Rossum', birth_date=datetime.date(1956, 1, 31), likes_python=True))


The main differences between the two can be summarized as followed:

| Property     | Dataclass                                | NamedTuple |
|--------------|------------------------------------------|------------|
| Mutable      | Yes (but can be restricted)              | No         |
| Customizable | Yes (repr, hash, mutability, comparison) | No         |
| Unpackable   | No                                       | Yes        |

As a rule of thumb, 
- if you would have used a tuple but naming the fields make it easier to manipulate, go for a NamedTuple. For instance, when returning several values at the end of a functions, or when creating a DataFrame:
- if you have many fields and some logics, go for a Dataclass;
- if you deal with inheritance, go for a Dataclass;
- exercize judgement for the gray in-between.

> There is an alternative syntax which does not need to inherint from `NamedTuple`: https://docs.python.org/3.9/library/collections.html#collections.namedtuple (it was the original syntax, although I personnally feel it is a bid unweildy).

Best practices
--------------

When writing classes, there are a few principles that are worth following:
- [ ] stick to Python conventions (eg. case, protected/private attributes, action/actor names);
- [ ] give clear and descriptive names (*); 
- [ ] make anything protected by default;
- [ ] provide an evaluable repr if possible;
- [ ] inheritance is a great power, blabla responsibility :spider: (use it wisely);
- [ ] consider returning self to chain calls;
- [ ] type (production) code: well-typed and explicit variable names will drastically cut down the what-the-f*ck factor.
- [ ] **never** use a mutable object as default value


> (*) Concise is best, long is better than fuzzy (tips: remember the single-responsibility principle). A good name prevents from writting three lines of doc.

Here is an example of typing and giving a good repr:

In [14]:
from __future__ import annotations

from typing import TypeVar

TCar = TypeVar("TCar", bound="Car")


class Car:
    def __init__(self, position: int = 0) -> None:
        self._position = position
        self._speed: int = 0

    def set_speed(self: TCar, speed: int) -> TCar:
        self._speed = speed
        return self

    def accelerate(self) -> None:
        self._speed += 1
    
    def decelerate(self) -> None:
        self._speed -= 1
    
    def move(self) -> None:
        self._position += self._speed

    
    def __repr__(self) -> str:
        r = f"{self.__class__.__qualname__}(position={self._position!r})"
        if self._speed != 0:
            r = f"{r}.set_speed({self._speed!r})"
        return r

    def __str__(self) -> str:
        return f"{self!r} @ __str__"
    
class LimitedCar(Car):
    def __init__(self, max_speed: int, position: int = 0) -> None:
        super().__init__(position)
        self._max_speed = max_speed

    def accelerate(self) -> None:
        if self._speed < self._max_speed:
            return super().accelerate()
        
    def __repr__(self) -> str:
        r = (
            f"{self.__class__.__qualname__}"
            f"("
            f"max_speed={self._max_speed!r}"
            f", "
            f"position={self._position!r}"
            f")"
        )
        if self._speed != 0:
            r = f"{r}.set_speed({self._speed!r})"
        return r
    

car = LimitedCar(10).set_speed(8)
car.accelerate()
print(repr(car))
print(car)

LimitedCar(max_speed=10, position=0).set_speed(9)
LimitedCar(max_speed=10, position=0).set_speed(9) @ __str__


> In Python >= 3.12, the typing module gives a `Self` type to explicitly state that the instance is returned (especially useful with sublcasses: avoid to create the bounded `TypeVar`)

Object creation
---------------

Design patterns are re-usable recipes to efficiently/elegantly solve recurring problems. One main issue of OOP is creating the right object, as is evident from the number of *creational design patterns*: abstract factory, builder, factory method, prototype, singleton, etc.

There are a couple of reasons why this is:
- the exact object needed is not known in advanced (eg. based on user input in a web interface);
- some part of the object specification are based on context (eg. how to handle NA and which quality check to perform is clearer when you know you are handling time series).

In any case, this sections deal with how to deal with object creation.

### (class) factory method

### Builder

### Prototype